In [1]:
import pandas as pd
import numpy as np
import os
from plotnine import ggplot, aes, geom_bar, coord_flip, theme, element_blank, labs, geom_text, theme_dark, scale_fill_manual, geom_col, guides, scale_x_discrete, element_text, scale_color_manual, ylim

In [2]:
data_file_path = r"C:\Users\useer\Downloads\WB_Sep_29_Oct_5_data_dump\WB_Report_data_2024-10-06.xlsx"

In [3]:
platforms = ['Facebook', 'Twitter', 'Instagram', 'YouTube']
category_strings = ['Party', 'Campaign', 'Leader']
color_coding = {'Facebook':'#366092', 'Twitter':'#6F6F6F', 'Instagram':'#C90784', 'YouTube':'#9E0000'}

In [4]:
entity_color_coding = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Color_mapping.xlsx").set_index('Name')['color_coding'].to_dict()

In [5]:
def left(df, column_name):    
    max_value = df[column_name].max()
    threshold = 0.6 * max_value
    filtered_df = df[df[column_name] < threshold] 
    return filtered_df

def right(df, column_name):    
    max_value = df[column_name].max()
    threshold = 0.6 * max_value
    filtered_df = df[df[column_name] >= threshold]
    return filtered_df

In [6]:
def mil_k_mapper(value):
    if value / 1000000 > 1:
        return str(round(value / 1000000, 1)) + 'M'
    else:
        return str(round(value / 1000, 1)) + 'k'

## Metric Charts

In [7]:
for platform in platforms:
    entity_color_coding = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Color_mapping.xlsx").set_index(platform)['color_coding'].to_dict()
    os.makedirs(rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform}", exist_ok=True)
    df_post = pd.read_excel(data_file_path, sheet_name = platform + '_all_stats_post')

    if platform == 'YouTube': 
        desired_order = list(reversed(df_post['presence_handle'].tolist()))
    else:
        id_mapping = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Party_mapping.xlsx")[platform].tolist()
        desired_order = list(reversed([profile for profile in id_mapping if profile in df_post['presence_handle'].tolist()]))

    df_post = df_post.fillna(0)
    df_post['presence_handle'] = pd.Categorical(df_post['presence_handle'], categories = desired_order, ordered=True)

    if platform == 'YouTube':
        df_post['Total Video Views kM_mapping'] = df_post['Total Video Views'].apply(mil_k_mapper)
        df_post['Avg Video Views kM_mapping'] = df_post['Avg Video Views'].apply(mil_k_mapper)
        df_post['Followers kM_mapping'] = df_post['Followers'].apply(mil_k_mapper)
        df_post[['Total Posts',	'count_of_active_days', 'Avg Video Views']] = df_post[['Total Posts', 'count_of_active_days', 'Avg Video Views']].astype(int)

    elif platform == 'Instagram':
        df_post['Followers kM_mapping'] = df_post['Followers'].apply(mil_k_mapper)
        df_post[['Total Posts',	'count_of_active_days']] = df_post[['Total Posts',	'count_of_active_days']].astype(int)        

    else:
        df_post['Avg Video Views kM_mapping'] = df_post['Avg Video Views'].apply(mil_k_mapper)
        df_post['Followers kM_mapping'] = df_post['Followers'].apply(mil_k_mapper)
        df_post[['Total Posts',	'count_of_active_days', 'Avg Video Views']] = df_post[['Total Posts', 'count_of_active_days', 'Avg Video Views']].astype(int)

    for category in category_strings:
        mapping_df = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Party_mapping.xlsx")
        mapping_order = mapping_df[mapping_df['mapping'] == category].dropna(subset = platform)[platform].tolist()
        if category == 'Leader':
            if platform == 'YouTube':
                fig_size = (3, 4.5)
            else:
                fig_size = (3, 6)
        else:
            fig_size = (3, 1.2)

        os.makedirs(rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform}\{category}", exist_ok=True)
        df = df_post[df_post['mapping'] == category]

        for column_name in df.select_dtypes(exclude=['object', 'category']).columns.tolist():
            plot = (
                ggplot(df, aes(x='presence_handle', y = column_name)) +
                geom_bar(stat='identity', color = color_coding[platform], fill=color_coding[platform]) +
                coord_flip() +  
                labs(x='', y='') +  
                theme_dark() +
                theme(
                    panel_background=element_blank(),  
                    plot_background=element_blank(),
                    panel_grid_major=element_blank(),
                    panel_grid_minor=element_blank(),
                    axis_text_x=element_blank(),  
                    axis_text_y=element_blank(),  
                    axis_ticks_major_x=element_blank(),
                    axis_ticks_minor_x=element_blank(),
                    axis_ticks=element_blank(),
                    figure_size = fig_size
                )
            )
            
            id_name_mapping = mapping_df[mapping_df['mapping'] == category].dropna(subset = platform).set_index(platform)['Name'].to_dict()
            plot += scale_x_discrete(labels = list(reversed([id_name_mapping[key] for key in mapping_df[mapping_df['mapping'] == category].dropna(subset = platform)[platform].tolist()])))

            if column_name in ['Total Video Views','Avg Video Views', 'Followers']:
                plot += geom_text(mapping=aes("presence_handle",column_name,label=column_name + ' kM_mapping'), data=right(df, column_name),size=8, ha='right',format_string=' {} ', family = 'Bahnschrift')
                plot += geom_text(mapping=aes("presence_handle",column_name,label=column_name + ' kM_mapping'), data=left(df, column_name),size=8, ha='left',format_string=' {} ', family = 'Bahnschrift')
            elif column_name in ['Total Posts', 'count_of_active_days']:
                plot += geom_text(mapping=aes("presence_handle",column_name,label=column_name), data=right(df, column_name),size=8, ha='right',format_string=' {:,} ', family = 'Bahnschrift')
                plot += geom_text(mapping=aes("presence_handle",column_name,label=column_name), data=left(df, column_name),size=8, ha='left',format_string=' {:,} ', family = 'Bahnschrift')
            else:
                plot += geom_text(mapping=aes("presence_handle",column_name,label=column_name), data=right(df, column_name),size=8, ha='right',format_string=' {:,.1f} ', family = 'Bahnschrift')
                plot += geom_text(mapping=aes("presence_handle",column_name,label=column_name), data=left(df, column_name),size=8, ha='left',format_string=' {:,.1f} ', family = 'Bahnschrift')

            if category != 'Party':
                if column_name in ['Followers', 'Avg. Post Per Day', 'impressions', 'Total Engagement']:
                    plot += theme(axis_text_y = element_text(color = list(reversed([entity_color_coding[key] if not pd.isna(key) else '#6F6F6F' for key in [x for x in mapping_order if not pd.isna(x)]])), family = 'Bahnschrift'), figure_size=(fig_size[0] * 1.68, fig_size[1]))

            elif column_name in ['Followers', 'Total Engagement']:
                plot += theme(axis_text_y = element_text(color = list(reversed([entity_color_coding[key] if not pd.isna(key) else '#6F6F6F' for key in [x for x in mapping_order if not pd.isna(x)]])), family = 'Bahnschrift'), figure_size=(fig_size[0] * 1.3, fig_size[1]))

            plot.save(filename=rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform}\{category}\{column_name}.png", dpi=300, format='png')    

f:\Projects\PPTX_env\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 3 x 1.2 in image.
f:\Projects\PPTX_env\lib\site-packages\plotnine\ggplot.py:607: PlotnineWarning: Filename: F:\Projects\PPTX_env\WB_weekly\bar_chart_images\Facebook\Party\Avg Video Views.png
f:\Projects\PPTX_env\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 3.9000000000000004 x 1.2 in image.
f:\Projects\PPTX_env\lib\site-packages\plotnine\ggplot.py:607: PlotnineWarning: Filename: F:\Projects\PPTX_env\WB_weekly\bar_chart_images\Facebook\Party\Total Engagement.png
f:\Projects\PPTX_env\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 3 x 1.2 in image.
f:\Projects\PPTX_env\lib\site-packages\plotnine\ggplot.py:607: PlotnineWarning: Filename: F:\Projects\PPTX_env\WB_weekly\bar_chart_images\Facebook\Party\Total Posts.png
f:\Projects\PPTX_env\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 3.9000000000000004 x 1.2 in image.
f:\Projects\PPTX_env\lib\site-

## Percentage Change Charts

In [8]:
for platform in platforms:
    entity_color_coding = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Color_mapping.xlsx").set_index(platform)['color_coding'].to_dict()
    os.makedirs(rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform}_delta_metrics", exist_ok=True)
    df_post = pd.read_excel(data_file_path, sheet_name = platform + '_all_stats_post')
    df_pre = pd.read_excel(data_file_path, sheet_name = platform + '_all_stats_pre_')

    if platform == 'YouTube': 
        desired_order = list(reversed(df_post['presence_handle'].tolist()))
    else:
        id_mapping = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Party_mapping.xlsx")[platform].tolist()
        desired_order = list(reversed([profile for profile in id_mapping if profile in df_post['presence_handle'].tolist()]))

    df_post = df_post.sort_values('presence_handle', key=lambda x: x.str.lower())
    df_pre = df_pre.sort_values('presence_handle', key=lambda x: x.str.lower())

    df_post_nums = df_post[df_post.select_dtypes(exclude=['object', 'category']).columns.tolist()].fillna(0)
    df_pre_nums = df_pre[df_post.select_dtypes(exclude=['object', 'category']).columns.tolist()].fillna(0)
    post_array = df_post_nums.to_numpy()
    pre_array = df_pre_nums.to_numpy()

    epsilon = 0.00000001
    pre_array = np.where(pre_array == 0, epsilon, pre_array)

    result_array = ((post_array / pre_array) - 1) * 100

    result_array[result_array > 1000000] = 0.0001           # if the pre array value is 0
    result_array[result_array == -100] = -0.0001            # if the post array value is 0

    delta = pd.DataFrame(result_array, columns = df_post.select_dtypes(exclude=['object', 'category']).columns.tolist())
    delta = delta.fillna(0)

    delta['presence_handle'] = df_post['presence_handle'].tolist()
    delta['mapping'] = df_post['mapping'].tolist()
    delta['presence_handle'] = pd.Categorical(delta['presence_handle'], categories = desired_order, ordered=True)

    for category in category_strings:
        mapping_order = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Party_mapping.xlsx")
        mapping_order = mapping_order[mapping_order['mapping'] == category].dropna(subset = platform)[platform].tolist()
        
        if category == 'Leader':
            if platform == 'YouTube':
                fig_size = (3, 4.5)
            else:
                fig_size = (3, 6)
        else:
            fig_size = (3, 1.2)

        os.makedirs(rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform}_delta_metrics\{category}", exist_ok=True)
        df = delta[delta['mapping'] == category]

        for column_name in df.select_dtypes(exclude=['object', 'category']).columns.tolist():
            df['label_text'] = df[column_name].apply(lambda x: '-' if (x == 0.0001) or (x == -0.0001) else f'{x:.1f}%')
            df['label_color'] = df[column_name].apply(lambda x: 'green' if x == 0.0001 else ('red' if x == -0.0001 else 'black'))

            conditions = [
                df[column_name] < 0,
                df[column_name] == 0,
                df[column_name] > 0
            ]

            values = ['red', 'grey', 'green']

            df['Bar_color_col'] = np.select(conditions, values, default='')
            df[column_name] = df[column_name].replace(0, (0.01 * df[column_name].abs().max()))

            plot = (
                ggplot(df, aes(x='presence_handle', y=column_name)) +
                geom_col(color = None) +
                coord_flip() +
                labs(x='', y='') +
                theme_dark() +
                theme(
                panel_background=element_blank(),
                plot_background=element_blank(),
                panel_grid_major=element_blank(),
                panel_grid_minor=element_blank(),
                axis_text_x=element_blank(),
                axis_text_y=element_blank(),
                axis_ticks_major_x=element_blank(),
                axis_ticks_minor_x=element_blank(),
                axis_ticks=element_blank(),
                figure_size=fig_size
            ) +
            guides(fill=False)
            )

            plot += geom_col(
                aes(x='presence_handle', y=column_name, fill=(df['Bar_color_col'])),
                color=None
            )

            id_name_mapping = mapping_df[mapping_df['mapping'] == category].dropna(subset = platform).set_index(platform)['Name'].to_dict()
            plot += scale_x_discrete(labels = list(reversed([id_name_mapping[key] for key in mapping_df[mapping_df['mapping'] == category].dropna(subset = platform)[platform].tolist()])))
            
            plot += scale_fill_manual(values={'red': 'red', 'green': 'green', 'grey':'grey'})               # dict = {label : color}

            if df[column_name].max() == df[column_name].min():
                plot += geom_text(mapping=aes("presence_handle", column_name, label = 'label_text'), data = df, size = 8, ha='left', family = 'Bahnschrift', nudge_y = 0.005)
                df[column_name] = df[column_name].replace(0, 0.01)
                plot +=  ylim(-0.5, 0.5) 
                plot += guides(color=False)      

            else:
                plot += geom_text(mapping=aes("presence_handle",column_name,label='label_text', color='label_color'), data=left(df[df['label_color'] == 'black'], column_name),size = 8, ha='left', family = 'Bahnschrift')
                plot += geom_text(mapping=aes("presence_handle",column_name,label='label_text', color='label_color'), data=right(df[df['label_color'] == 'black'], column_name),size = 8, ha='right', family = 'Bahnschrift')

                plot += geom_text(mapping=aes("presence_handle",column_name,label='label_text', color='label_color'), data=left(df[df['label_color'] != 'black'], column_name),size = 12, ha='left', family = 'Bahnschrift')
                plot += geom_text(mapping=aes("presence_handle",column_name,label='label_text', color='label_color'), data=right(df[df['label_color'] != 'black'], column_name),size = 12, ha='right', family = 'Bahnschrift')
                plot += scale_color_manual(values={'green': '#008000', 'red': '#FF0000', 'black': '#000000'})
                plot += guides(color=False)   

            if category == 'Party':
                if column_name in ['Followers', 'Total Engagement']:
                    plot += theme(axis_text_y = element_text(color = list(reversed([entity_color_coding[key] if not pd.isna(key) else '#6F6F6F' for key in [x for x in mapping_order if not pd.isna(x)]])), family = 'Bahnschrift'), figure_size=(fig_size[0] * 1.3, fig_size[1]))

            plot.save(filename=rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform}_delta_metrics\{category}\{column_name}.png", dpi=300, format='png')

C:\Users\useer\AppData\Local\Temp\ipykernel_8804\2755778510.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\useer\AppData\Local\Temp\ipykernel_8804\2755778510.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\useer\AppData\Local\Temp\ipykernel_8804\2755778510.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

## Post Distribution Charts

In [9]:
post_dist_path = r"C:\Users\useer\Downloads\WB_Sep_29_Oct_5_data_dump\WB_Report_data_2024-10-06_post_dist.xlsx"

for sheet_name in pd.ExcelFile(post_dist_path).sheet_names:
    df_parent = pd.read_excel(post_dist_path, sheet_name = sheet_name)
    id_mapping = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Color_mapping.xlsx")[sheet_name].tolist()

    desired_order = list(reversed([profile for profile in id_mapping if profile in df_parent['presence_handle'].tolist()]))
    desired_order = [x for x in desired_order if pd.notna(x)]
    df_parent = df_parent.dropna(subset = ['presence_handle'])
    df_parent = df_parent.apply(lambda col: col.fillna('video') if col.name == 'post_type' else col.fillna(0))
    df_parent['presence_handle'] = pd.Categorical(df_parent['presence_handle'], categories = desired_order, ordered=True)

    for category in list(df_parent['mapping'].unique()):
        if category == 'Leader':
            fig_size = (3, 6)
        else:
            fig_size = (3, 1.2)

        df = df_parent[df_parent['mapping'] == category]
        df = df.groupby('presence_handle', group_keys=False).apply(lambda x: x.sort_values(by='post_type'))

        group_totals = df.groupby('presence_handle')['count'].transform('sum')

        df['cumulative_sum'] = df.groupby('presence_handle')['count'].cumsum()
        df['label_position'] = group_totals - (df['cumulative_sum'] - df['count'] / 2)

        df['adjusted_count'] = df['count'].apply(lambda x: (0.002 * df['count'].max()) if x == 0 else x)
        df['adjusted_label_position'] = df.apply(lambda row: row['adjusted_count'] + (0.01 * df['count'].max()) if row['count'] == 0 else row['label_position'], axis=1)

        plot = (
            ggplot(df, aes(x='presence_handle', y='adjusted_count', fill='post_type')) +
            geom_bar(stat='identity', color=None) +             # identity : stacked bar chart 
            geom_text(
            aes(label=df['count'].astype(int), 
                y='adjusted_label_position'),
            color='black', size=7, family = 'Bahnschrift'
            ) +
            coord_flip() +
            labs(x='', y='') +  
            theme_dark() +
            theme(
                panel_background=element_blank(), 
                plot_background=element_blank(),
                panel_grid_major=element_blank(),
                panel_grid_minor=element_blank(),
                axis_text_x=element_blank(),
                axis_text_y=element_blank(), 
                axis_ticks_major_x=element_blank(),
                axis_ticks_minor_x=element_blank(),
                axis_ticks=element_blank(),
                figure_size=fig_size
            ) 
        + guides(fill=False)
        )
        plot.save(filename=rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{sheet_name}\{category}\Post_distribution.png", dpi=300, format='png')

C:\Users\useer\AppData\Local\Temp\ipykernel_8804\3688198940.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
C:\Users\useer\AppData\Local\Temp\ipykernel_8804\3688198940.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
C:\Users\useer\AppData\Local\Temp\ipykernel_8804\3688198940.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence th